In [77]:
folder = './free_movements/'

In [78]:
import os
import re
import numpy as np
import pandas as pd

In [79]:
leap_path = os.path.join(folder, 'leap.txt')
with open(leap_path) as f:
    leap_str = f.read()

In [80]:
leap_frames = leap_str.split('Frame id: ')
len(leap_frames)

3366

In [81]:
def process_leap_frame(frame_str, verbose=False):
    out_struct = {}
    
    if verbose:
        print(frame_str)
    
    re_out = re.findall('(\d+), timestamp: (\d+), hands: (\d+), fingers: (\d+), tools: (\d+), gestures: (\d+)', frame_str)
    assert(len(re_out) <= 1), "Multiple frames found" + frame_str
    
    if len(re_out)==0:
        return out_struct
    
    frame_id, timestamp, hands, fingers, tools, gestures = np.array(re_out[0]).astype(int)
    
    out_struct = {'frame_id': frame_id, 
                  'timestamp': timestamp, 
                  'hands': hands, 
                  'fingers': fingers, 
                  'tools': tools, 
                  'gestures': gestures}
    
    assert(hands<=1), f'Frame ID:{frame_id} - only one hand could by parsed at the moment'
    
    if hands==1:
        hands_reg = '\s+(.+) hand, id (\d+), position: (\(.+\))\\n'+\
                    '\s+pitch: (.+) degrees, roll: (.+) degrees, yaw: (.+) degrees\\n'+\
                    '\s+Arm direction: (\(.+\)), wrist position: (\(.+\)), elbow position: (\(.+\))\\n'+\
                    '([\s\S]+)'

        fingers_reg = '(\w+) finger, id: (.+), length: (.+)mm, width: (.+)mm\\n'+\
                      '.+start: (\(.+\)), end.+\\n'+\
                      '.+start: (\(.+\)), end.+\\n'+\
                      '.+start: (\(.+\)), end.+\\n'+\
                      '.+start: (\(.+\)), end: (\(.+\)).+\\n'

        hands_info = re.findall(hands_reg, frame_str)

        hand_label, hand_id, hand_position, pitch, roll, yaw, arm_dir, wrist_pos, elbow_pos, dig_str = hands_info[0]
        out_struct['hand'+'_label'] = hand_label
        out_struct['hand'+'_id'] = hand_id
#         print(hand_position)
        out_struct['hand'+'_pos'] = eval(hand_position)
        out_struct['hand'+'_pitch'] = float(pitch)
        out_struct['hand'+'_roll'] = float(roll)
        out_struct['hand'+'_yaw'] = float(yaw)
        out_struct['hand'+'_arm_dir'] = eval(arm_dir)
        out_struct['hand'+'_wrist_pos'] = eval(wrist_pos)
        out_struct['hand'+'_elbow_pos'] = eval(elbow_pos)

        fingers_info = re.findall(fingers_reg, dig_str)

        for f_label, f_id, f_l, f_w, f1, f2, f3, f4, f5 in fingers_info:
            full_f_label = f'{f_label}'
            out_struct[full_f_label+'_id'] = f_id
            out_struct[full_f_label+'_length'] = float(f_l)
            out_struct[full_f_label+'_width'] = float(f_w)
            out_struct[full_f_label+'_1'] = eval(f1)
            out_struct[full_f_label+'_2'] = eval(f2)
            out_struct[full_f_label+'_3'] = eval(f3)
            out_struct[full_f_label+'_4'] = eval(f4)
            out_struct[full_f_label+'_5'] = eval(f5)

    
    return out_struct
    
process_leap_frame(leap_frames[-320])

{'frame_id': 566506,
 'timestamp': 79342671398,
 'hands': 1,
 'fingers': 5,
 'tools': 0,
 'gestures': 0,
 'hand_label': 'Left',
 'hand_id': '186',
 'hand_pos': (109.585, 115.344, 166.866),
 'hand_pitch': 26.489571,
 'hand_roll': 64.548956,
 'hand_yaw': 59.694106,
 'hand_arm_dir': (0.9167, 0.0805504, -0.391373),
 'hand_wrist_pos': (63.8588, 96.6966, 186.618),
 'hand_elbow_pos': (-176.775, 75.5521, 289.353),
 'Thumb_id': '1860',
 'Thumb_length': 50.668617,
 'Thumb_width': 19.687469,
 'Thumb_1': (64.8671, 119.084, 206.298),
 'Thumb_2': (64.8671, 119.084, 206.298),
 'Thumb_3': (113.161, 123.231, 203.911),
 'Thumb_4': (146.189, 121.343, 201.804),
 'Thumb_5': (168.607, 117.822, 200.148),
 'Index_id': '1861',
 'Index_length': 57.173882,
 'Index_width': 18.805471,
 'Index_1': (63.8033, 119.199, 183.277),
 'Index_2': (127.127, 144.568, 161.773),
 'Index_3': (168.673, 146.366, 157.854),
 'Index_4': (187.364, 141.532, 171.251),
 'Index_5': (193.524, 136.195, 185.725),
 'Middle_id': '1862',
 'Midd

In [82]:
df_leap = pd.DataFrame(list(filter(len, map(process_leap_frame, leap_frames))))
df_leap.timestamp -= df_leap.timestamp.iloc[0]
df_leap.dropna(subset={'hand_id'}, inplace=True)
df_leap.set_index('frame_id', drop=True, inplace=True)

df_leap['time_s'] = df_leap.timestamp/1e6
len(df_leap)

3016

In [83]:
df_leap

,timestamp,hands,fingers,tools,gestures,hand_label,hand_id,hand_pos,hand_pitch,hand_roll,...,Ring_5,Pinky_id,Pinky_length,Pinky_width,Pinky_1,Pinky_2,Pinky_3,Pinky_4,Pinky_5,time_s
frame_id,,,,,,,,,,,,,,,,,,,,,
557515,2920066,1,5,0,0,Right,185,"(25.3163, 327.678, 77.7134)",42.331320,5.753457,...,"(10.0534, 288.848, 65.6281)",1854,48.322369,15.361754,"(-6.17804, 311.28, 111.322)","(48.7613, 314.194, 104.22)","(50.2032, 292.142, 78.6087)","(32.0152, 290.395, 74.5759)","(16.5869, 294.947, 78.2053)",2.920066
557516,3024300,1,5,0,0,Right,185,"(28.5266, 298.259, 37.7801)",34.071528,5.551091,...,"(13.6161, 257.939, 31.5569)",1854,48.322369,15.361754,"(-2.07539, 286.345, 73.9999)","(52.6695, 288.9, 65.4138)","(55.4234, 263.501, 43.2427)","(37.3983, 260.443, 39.26)","(21.8189, 265.002, 42.1606)",3.024300
557517,3128610,1,5,0,0,Right,185,"(20.0054, 306.169, -8.50233)",68.405985,24.429022,...,"(17.1621, 267.626, -28.9509)",1854,48.322369,15.361754,"(-3.20197, 274.825, 20.9819)","(47.8968, 295.509, 14.7901)","(56.0168, 280.793, -14.5659)","(39.3202, 274.18, -19.8208)","(23.3528, 272.876, -15.9142)",3.128610
557520,3155753,1,5,0,0,Right,185,"(16.6855, 304.792, -10.6674)",70.089093,25.061499,...,"(18.7249, 268.186, -35.2816)",1854,48.322369,15.361754,"(-5.88471, 272.041, 17.7624)","(44.9871, 293.322, 11.7279)","(54.3487, 280.401, -18.0998)","(38.069, 273.626, -24.3596)","(22.0405, 271.765, -20.9621)",3.155753
557523,3182853,1,5,0,0,Right,185,"(11.4403, 301.547, -10.6246)",69.382524,24.850228,...,"(15.2882, 266.682, -36.8881)",1854,48.322369,15.361754,"(-8.71258, 267.328, 17.8924)","(41.1452, 290.337, 10.0163)","(51.1573, 279.171, -20.3048)","(35.2411, 271.805, -26.8264)","(19.3836, 269.118, -23.1873)",3.182853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566493,85741759,1,5,0,0,Left,186,"(67.2564, 156.8, 139.972)",28.080407,41.744044,...,"(142.672, 125.007, 132.356)",1864,49.107586,15.611376,"(24.6891, 132.21, 147.181)","(65.3678, 135.082, 108.257)","(99.3901, 132.807, 103.889)","(113.271, 125.75, 114.803)","(118.128, 118.275, 128.993)",85.741759
566496,85768904,1,5,0,0,Left,186,"(73.0325, 141.878, 145.165)",30.121111,47.819258,...,"(151.352, 116.872, 146.081)",1864,49.107586,15.611376,"(32.4379, 114.075, 152.071)","(75.5256, 117.078, 115.842)","(109.889, 117.338, 114.907)","(122.916, 112.581, 127.916)","(126.627, 106.993, 143.273)",85.768904
566500,85805084,1,5,0,0,Left,186,"(81.2683, 135.659, 162.535)",20.116615,46.564511,...,"(158.774, 104.669, 151.52)",1864,49.107586,15.611376,"(39.4434, 111.634, 174.455)","(79.0614, 109.487, 134.406)","(113.086, 106.892, 130.236)","(126.919, 101.657, 142.186)","(131.661, 96.4577, 157.395)",85.805084


In [84]:
df_leap.to_csv(os.path.join(folder, 'leap_preprocessed.csv'), sep='\t')

In [85]:
from mpl_toolkits import mplot3d
from matplotlib import pyplot as plt
%matplotlib ipympl

row = df_leap[['hand_pos', 'hand_wrist_pos', 'hand_elbow_pos', 
                 'Thumb_1', 
                 'Thumb_2', 
                 'Thumb_3', 
                 'Thumb_4', 
                 'Thumb_5',
                 'Index_1', 
                 'Index_2', 
                 'Index_3', 
                 'Index_4', 
                 'Index_5',
                 'Middle_1', 
                 'Middle_2', 
                 'Middle_3', 
                 'Middle_4', 
                 'Middle_5',
                 'Ring_1', 
                 'Ring_2', 
                 'Ring_3', 
                 'Ring_4', 
                 'Ring_5',
                 'Pinky_1', 
                 'Pinky_2', 
                 'Pinky_3', 
                 'Pinky_4', 
                 'Pinky_5',
                ]].iloc[0]

fig = plt.figure()
ax = plt.axes(projection='3d')
hand_line = ax.plot(*(np.stack(row[['hand_pos', 'hand_wrist_pos']]).T), '.-', c='k', ms=25, lw=2)[0]
finger_lines = []
for ifinger, finger in enumerate(['Thumb', 'Index', 'Middle', 'Ring', 'Pinky']):    
    l = ax.plot(*(np.stack(row[[f'{finger}_1', 
                            f'{finger}_2', 
                            f'{finger}_3', 
                            f'{finger}_4', 
                            f'{finger}_5']]).T), '.-', ms=10, c=f'C{ifinger}', lw=1)
    finger_lines.append(l[0])
    
plt.legend(['Hand', 'Thumb', 'Index', 'Middle', 'Ring', 'Pinky'])
    
dd = df_leap[['hand_pos', 'hand_wrist_pos', 'hand_elbow_pos', 
                 'Thumb_1', 
                 'Thumb_2', 
                 'Thumb_3', 
                 'Thumb_4', 
                 'Thumb_5',
                 'Index_1', 
                 'Index_2', 
                 'Index_3', 
                 'Index_4', 
                 'Index_5',
                 'Middle_1', 
                 'Middle_2', 
                 'Middle_3', 
                 'Middle_4', 
                 'Middle_5',
                 'Ring_1', 
                 'Ring_2', 
                 'Ring_3', 
                 'Ring_4', 
                 'Ring_5',
                 'Pinky_1', 
                 'Pinky_2', 
                 'Pinky_3', 
                 'Pinky_4', 
                 'Pinky_5',
                ]].values

    
xp, yp, zp = np.stack(np.stack(dd, axis=0).reshape(-1), axis=0).T
plt.xlim(xp.min(), xp.max())
plt.ylim(yp.min(), yp.max())
ax.set_zlim(zp.min(), zp.max())

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.title(f"{df_leap['time_s'].iloc[0]} seconds")
    
plt.tight_layout()


def update_lines(frame_num):
    row = df_leap[['hand_pos', 'hand_wrist_pos', 
                 'Thumb_1', 
                 'Thumb_2', 
                 'Thumb_3', 
                 'Thumb_4', 
                 'Thumb_5',
                 'Index_1', 
                 'Index_2', 
                 'Index_3', 
                 'Index_4', 
                 'Index_5',
                 'Middle_1', 
                 'Middle_2', 
                 'Middle_3', 
                 'Middle_4', 
                 'Middle_5',
                 'Ring_1', 
                 'Ring_2', 
                 'Ring_3', 
                 'Ring_4', 
                 'Ring_5',
                 'Pinky_1', 
                 'Pinky_2', 
                 'Pinky_3', 
                 'Pinky_4', 
                 'Pinky_5',
                ]].iloc[frame_num]
    
    plt.title(f"{frame_num} - {df_leap['time_s'].iloc[frame_num]:.3f} seconds")
    hand_line.set_data_3d(*(np.stack(row[['hand_pos', 'hand_wrist_pos']]).T))
    for l, finger in (zip(finger_lines, ['Thumb', 'Index', 'Middle', 'Ring', 'Pinky'])):    
        l.set_data_3d(*(np.stack(row[[f'{finger}_1', 
                                f'{finger}_2', 
                                f'{finger}_3', 
                                f'{finger}_4', 
                                f'{finger}_5']]).T))    
    

# Creating the Animation object
import matplotlib.animation as animation
line_ani = animation.FuncAnimation(fig, update_lines, frames=len(df_leap), interval=10, repeat=False)

line_ani.save(os.path.join(folder, 'leap.mp4'), fps=int(np.round((1/(df_leap.time_s.diff().mode())))), dpi=100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …